In [1]:
import pandas as pd

In [2]:
df_PCPN = pd.read_csv('PCPN.csv')
df_TMAX = pd.read_csv('TMAX.csv')
df_TAVG = pd.read_csv('TAVG.csv')
df_TMAX = df_TMAX.rename(columns={'Unnamed: 0': 'ID', 'Unnamed: 1': 'Janvier', 'Unnamed: 2': 'Février', 'Unnamed: 3': 'Mars', 'Unnamed: 4': 'Avril', 'Unnamed: 5': 'Mai', 'Unnamed: 6': 'Juin', 'Unnamed: 7': 'Juillet', 'Unnamed: 8': 'Août', 'Unnamed: 9': 'Septembre', 'Unnamed: 10': 'Octobre', 'Unnamed: 11': 'Novembre', 'Unnamed: 12': 'Décembre'})

In [3]:
#Création des DataFrames fusionnables:

def cleaning_df(df,string):
    
    df['ID'] = df.ID.astype(str).str.zfill(11)
    
    df['Etat'] = df.ID.astype(str).apply(lambda x:x[:2])
    
    df['Comté'] = df.ID.astype(str).apply(lambda x:x[2:5])

    df['Année'] = df.ID.astype(str).apply(lambda x:x[-4:])
    df = df.drop('ID', axis = 1)
    
    Liste_mois = ['Janvier','Février','Mars','Avril','Mai','Juin','Juillet','Août','Septembre','Octobre','Novembre','Décembre']
    df = pd.melt(df,['Comté','Année','Etat'],Liste_mois)


    df = df.rename(columns={'value':string,'variable' : 'Mois'})
    
    df = df.loc[(df['Année'].astype(int) > 1991) & (df['Année'].astype(int) < 2016)]
    
    return df


df_PCPN_clean = cleaning_df(df_PCPN,'PCPN')
df_TMAX_clean = cleaning_df(df_TMAX,'TMAX')
df_TAVG_clean = cleaning_df(df_TAVG,'TAVG')




In [4]:
df_TMAX_clean.head()

,Comté,Année,Etat,Mois,TMAX
97,001,1992,01,Janvier,54.5
98,001,1993,01,Janvier,59.3
99,001,1994,01,Janvier,52.2
100,001,1995,01,Janvier,57.0
101,001,1996,01,Janvier,56.0


In [5]:
#Fusion des trois DataFrames:

df_deux = pd.merge(left =df_TMAX_clean, right=df_TAVG_clean, how="inner", left_on=['Comté','Année','Etat','Mois'], right_on=['Comté','Année','Etat','Mois'])
df_ALL = pd.merge(left =df_deux, right=df_PCPN_clean, how="inner", left_on=['Comté','Année','Etat','Mois'], right_on=['Comté','Année','Etat','Mois'])


,Comté,Année,Etat,Mois,TMAX,TAVG,PCPN
0,001,1992,01,Janvier,54.5,44.6,5.62
1,001,1993,01,Janvier,59.3,49.7,6.08
2,001,1994,01,Janvier,52.2,41.7,3.54
3,001,1995,01,Janvier,57.0,46.4,4.95
4,001,1996,01,Janvier,56.0,44.0,7.04
5,001,1997,01,Janvier,57.0,47.0,6.20
6,001,1998,01,Janvier,58.9,48.5,7.28
7,001,1999,01,Janvier,63.4,50.5,8.46
8,001,2000,01,Janvier,58.9,47.9,5.52
9,001,2001,01,Janvier,53.8,42.2,4.18


In [6]:
#Création d'un DataFrame exportable:
df_ALL.to_csv('df_TMAX_TAVG_PCPN')


In [41]:
df_init = pd.read_csv('df_initial.csv',usecols=['FIRE_YEAR','DISCOVERY_DATE','FIPS_CODE','STATE','FIRE_SIZE','COUNTY'])
df_init.head()


def julian_to_gregorian(julian_date):
    return pd.to_datetime(julian_date, origin='julian', unit='D').strftime('%d/%m/%Y')
# Appliquer la conversion à la colonne DISCOVERY_DATE
df_init['DISCOVERY_DATE'] = df_init['DISCOVERY_DATE'].apply(julian_to_gregorian)

C:\Users\pierr\AppData\Local\Temp\ipykernel_16976\1380194724.py:1: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  df_init = pd.read_csv('df_initial.csv',usecols=['FIRE_YEAR','DISCOVERY_DATE','FIPS_CODE','STATE','FIRE_SIZE','COUNTY'])


,FIRE_YEAR,DISCOVERY_DATE,FIRE_SIZE,STATE,COUNTY,FIPS_CODE
0,2005,02/02/2005,0.10,CA,63.0,63.0
1,2004,12/05/2004,0.25,CA,61.0,61.0
2,2004,31/05/2004,0.10,CA,17.0,17.0
3,2004,28/06/2004,0.10,CA,3.0,3.0
4,2004,28/06/2004,0.10,CA,3.0,3.0


In [43]:
df_init.DISCOVERY_DATE = pd.to_datetime(df_init.DISCOVERY_DATE).dt.month

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '31/05/2004' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '28/06/2004' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '30/06/2004' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '15/03/2005' in DD/MM/YYYY format. Provide format or specif

In [44]:
df_init = df_init.rename(columns={"DISCOVERY_DATE":"Mois","FIRE_YEAR":"Année",'STATE':'Etat','COUNTY':'Comté'})


In [46]:
#Création des listes en vue du remplacement. 
Liste_ID = ['01','02','03','04','05','06','07','08','09', 
            '10', '11','12','13','14','15','16','17','18','19', '20',
            '21','22','23','24','25','26','27','28','29', '30',
            '31','32','33','34','35','36','37','38','39', '40',
            '41','42','43','44','45','46','47','48','50']

Liste_Etats = ['AL','AZ','AR','CA','CO','CT','DE','FL','GA','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO',
              'MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY','AK']


df_init.FIPS_CODE = df_init.FIPS_CODE.fillna(999)
df_init.Etat = df_init.Etat.replace(Liste_Etats,Liste_ID)

df_init.Comté = df_init.Etat.astype(str) + '-' + round(df_init.FIPS_CODE,0).astype(int).astype(str).str.zfill(3)



df_init.head()

,Année,Mois,FIRE_SIZE,Etat,Comté,FIPS_CODE
0,2005,2,0.10,04,04-063,63.0
1,2004,12,0.25,04,04-061,61.0
2,2004,5,0.10,04,04-017,17.0
3,2004,6,0.10,04,04-003,3.0
4,2004,6,0.10,04,04-003,3.0


In [47]:
df_ALL.Mois = df_ALL.Mois.replace(['Janvier', 'Février', 'Mars', 'Avril', 'Mai', 'Juin', 'Juillet',
       'Août', 'Septembre', 'Octobre', 'Novembre', 'Décembre'],[1,2,3,4,5,6,7,8,9,10,11,12])

In [48]:
df_ALL['Etat_Comté'] = df_ALL.Etat.astype(str) + '-' + df_ALL.Comté.astype(str)

In [49]:
df_ALL.head()

,Comté,Année,Etat,Mois,TMAX,TAVG,PCPN,Etat_Comté
0,001,1992,01,1,54.5,44.6,5.62,01-001
1,001,1993,01,1,59.3,49.7,6.08,01-001
2,001,1994,01,1,52.2,41.7,3.54,01-001
3,001,1995,01,1,57.0,46.4,4.95,01-001
4,001,1996,01,1,56.0,44.0,7.04,01-001


In [55]:

df_ALL.Année = df_ALL.Année.astype('int64')


<class 'pandas.core.frame.DataFrame'>
Int64Index: 903456 entries, 0 to 903455
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Comté       903456 non-null  object 
 1   Année       903456 non-null  int64  
 2   Etat        903456 non-null  object 
 3   Mois        903456 non-null  int64  
 4   TMAX        903456 non-null  float64
 5   TAVG        903456 non-null  float64
 6   PCPN        903456 non-null  float64
 7   Etat_Comté  903456 non-null  object 
dtypes: float64(3), int64(2), object(3)
memory usage: 62.0+ MB


In [73]:
df_init.head()

,Année,Mois,FIRE_SIZE,Etat,Comté,FIPS_CODE
0,2005,2,0.10,04,04-063,63.0
1,2004,12,0.25,04,04-061,61.0
2,2004,5,0.10,04,04-017,17.0
3,2004,6,0.10,04,04-003,3.0
4,2004,6,0.10,04,04-003,3.0


In [79]:
df_init_groupé = df_init.groupby(['Année','Mois','Comté'],as_index=False).agg({'FIRE_SIZE':'sum','FIPS_CODE':'count'})
df_init_groupé.head()

,Année,Mois,Comté,FIRE_SIZE,FIPS_CODE
0,1992,1,01-999,65.1,13
1,1992,1,02-003,17.7,4
2,1992,1,02-009,1.0,1
3,1992,1,02-013,163.1,4
4,1992,1,02-015,57.5,2


In [88]:
df_FINAL = pd.merge(left = df_ALL, right = df_init_groupé, how = 'left', left_on = ['Année','Mois','Etat_Comté'], right_on = ['Année','Mois','Comté'])

In [106]:
#df_FINAL = df_FINAL.drop(['Comté_x','Comté_y'],axis=1)
#df_FINAL.FIPS_CODE = df_FINAL.FIPS_CODE.fillna(0).astype(int)
#df_FINAL.FIRE_SIZE = df_FINAL.FIRE_SIZE.fillna(0)
df_FINAL['Comté'] = df_FINAL.Etat_Comté.apply(lambda x:x[3:])
#df_FINAL = df_FINAL.rename(columns={'FIPS_CODE':'Nombre de feux','FIRE_SIZE':'Etendue totale des feux'})



,Année,Etat,Mois,TMAX,TAVG,PCPN,Etat_Comté,Etendue totale des feux,Nombre de feux,Comté
0,1992,01,1,54.5,44.6,5.62,01-001,0.00,0,001
1,1993,01,1,59.3,49.7,6.08,01-001,0.00,0,001
2,1994,01,1,52.2,41.7,3.54,01-001,19.60,3,001
3,1995,01,1,57.0,46.4,4.95,01-001,2.60,4,001
4,1996,01,1,56.0,44.0,7.04,01-001,0.00,0,001
5,1997,01,1,57.0,47.0,6.20,01-001,0.00,0,001
6,1998,01,1,58.9,48.5,7.28,01-001,0.00,0,001
7,1999,01,1,63.4,50.5,8.46,01-001,0.00,0,001
8,2000,01,1,58.9,47.9,5.52,01-001,0.00,0,001
9,2001,01,1,53.8,42.2,4.18,01-001,0.00,0,001


In [109]:
df_FINAL.to_csv('Dataframe_Final_Comtés_Mois.csv')